In [3]:
import numpy as np
import math
import pandas as pd
import cv2
import matplotlib.pyplot as plt

In [4]:
from struct import unpack
import gzip
import numpy as np
from numpy import zeros, uint8, float32

def get_labeled_data(imagefile, labelfile):
    """Read input-vector (image) and target class (label, 0-9) and return
       it as list of tuples.
    """
    # Open the images with gzip in read binary mode
    images = gzip.open(imagefile, 'rb')
    labels = gzip.open(labelfile, 'rb')

    # Read the binary data

    # We have to get big endian unsigned int. So we need '>I'

    # Get metadata for images
    images.read(4)  # skip the magic_number
    number_of_images = images.read(4)
    number_of_images = unpack('>I', number_of_images)[0]
    rows = images.read(4)
    rows = unpack('>I', rows)[0]
    cols = images.read(4)
    cols = unpack('>I', cols)[0]

    # Get metadata for labels
    labels.read(4)  # skip the magic_number
    N = labels.read(4)
    N = unpack('>I', N)[0]

    if number_of_images != N:
        raise Exception('number of labels did not match the number of images')

    # Get the data
    x = zeros((N, rows, cols), dtype=float32)  # Initialize numpy array
    y = zeros((N, 1), dtype=uint8)  # Initialize numpy array
    for i in range(N):
        #if i % 1000 == 0:
           # print("i: %i" % i)
        for row in range(rows):
            for col in range(cols):
                tmp_pixel = images.read(1)  # Just a single byte
                tmp_pixel = unpack('>B', tmp_pixel)[0]
                x[i][row][col] = tmp_pixel
        tmp_label = labels.read(1)
        y[i] = unpack('>B', tmp_label)[0]
    return (x, y)

In [5]:
def resize(imagefile):
    num, row, col=imagefile.shape
    imagefile_resized = np.zeros((num,20,20))
    for i in range (num):
        temp=np.zeros((28,28))
        temp = np.clip(imagefile[i], 0, 255)
        temp = np.clip(imagefile[i], 0, 255).astype('uint8')
        contours=cv2.findContours(temp,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h=cv2.boundingRect(cnt)    
        num_only=cnt[y:y+h,x:x+w]
        stretched =cv2.resize(num_only,(20,20))#     plt.imshow(stretched,cmap='Greys',interpolation='nearest')
#     plt.show()
#     print(stretched.shape)
        imagefile_resized[i]=stretched
    return imagefile_resized

In [6]:
def normalize(data):  ##将图片像素二值化
    m,n,x = np.array(data).shape
    data1 = np.zeros((m,n,x))
    for i in range(m):
        for j in range(n):
            for k in range(x):
                if data[i,j,k] > 127:
                    data1[i,j,k] = 1
                else:
                    data1[i,j,k] = 0
    return data1

In [7]:
def getMean(train_data, train_label):
    num, row ,col = train_data.shape
    numj = np.zeros(10)
    Mean = np.zeros((10,row,col))
    pyjk1 = np.zeros((10,row,col))
    pyj = np.zeros(10)
    for i in range(num):
        label = train_label[i]
        numj[label] = numj[label] + 1 
        if label == 0:
            for j in range(row):
                for k in range(col):
                    Mean[0][j][k] = Mean[0][j][k] + train_data[i][j][k]
        if label == 1:
            for j in range(row):
                for k in range(col):
                    Mean[1][j][k] = Mean[1][j][k] + train_data[i][j][k]
        if label == 2:
            for j in range(row):
                for k in range(col):
                    Mean[2][j][k] = Mean[2][j][k] + train_data[i][j][k]
        if label == 3:
            for j in range(row):
                for k in range(col):
                    Mean[3][j][k] = Mean[3][j][k] + train_data[i][j][k]
        if label == 4:
            for j in range(row):
                for k in range(col):
                    Mean[4][j][k] = Mean[4][j][k] + train_data[i][j][k]
        if label == 5:
            for j in range(row):
                for k in range(col):
                    Mean[5][j][k] = Mean[5][j][k] + train_data[i][j][k]
        if label == 6:
            for j in range(row):
                for k in range(col):
                    Mean[6][j][k] = Mean[6][j][k] + train_data[i][j][k]
        if label == 7:
            for j in range(row):
                for k in range(col):
                    Mean[7][j][k] = Mean[7][j][k] + train_data[i][j][k]
        if label == 8:
            for j in range(row):
                for k in range(col):
                    Mean[8][j][k] = Mean[8][j][k] + train_data[i][j][k]
        if label == 9:
            for j in range(row):
                for k in range(col):
                    Mean[9][j][k] = Mean[9][j][k] + train_data[i][j][k]
    mean = Mean.T/numj
    pyjk = (mean + 1) / (numj + 2)
    pyj = (numj + 1) / (num + 10)
    return  mean, pyj, pyjk

In [8]:
def getVar(train_data, train_label, mean):
    num, row ,col = train_data.shape
    numj = np.zeros(10)
    var = np.zeros((10,row,col))
    for i in range(num):
        label = train_label[i]
        numj[label] = numj[label] + 1 
        if label == 0:
            for j in range(row):
                for k in range(col):
                    var[0][j][k] = var[0][j][k] + math.pow((train_data[i][j][k]-mean.T[0][j][k]),2)
        if label == 1:
            for j in range(row):
                for k in range(col):
                    var[1][j][k] = var[1][j][k] + math.pow((train_data[i][j][k]-mean.T[1][j][k]),2)
        if label == 2:
            for j in range(row):
                for k in range(col):
                    var[2][j][k] = var[2][j][k] + math.pow((train_data[i][j][k]-mean.T[2][j][k]),2)
        if label == 3:
            for j in range(row):
                for k in range(col):
                    var[3][j][k] = var[3][j][k] + math.pow((train_data[i][j][k]-mean.T[3][j][k]),2)
        if label == 4:
            for j in range(row):
                for k in range(col):
                    var[4][j][k] = var[4][j][k] + math.pow((train_data[i][j][k]-mean.T[4][j][k]),2)
        if label == 5:
            for j in range(row):
                for k in range(col):
                    var[5][j][k] = var[5][j][k] + math.pow((train_data[i][j][k]-mean.T[5][j][k]),2)
        if label == 6:
            for j in range(row):
                for k in range(col):
                    var[6][j][k] = var[6][j][k] + math.pow((train_data[i][j][k]-mean.T[6][j][k]),2)
        if label == 7:
            for j in range(row):
                for k in range(col):
                    var[7][j][k] = var[7][j][k] + math.pow((train_data[i][j][k]-mean.T[7][j][k]),2)
        if label == 8:
            for j in range(row):
                for k in range(col):
                    var[8][j][k] = var[8][j][k] + math.pow((train_data[i][j][k]-mean.T[8][j][k]),2)
        if label == 9:
            for j in range(row):
                for k in range(col):
                    var[9][j][k] = var[9][j][k] + math.pow((train_data[i][j][k]-mean.T[9][j][k]),2)
    Var = var.T/numj
    return  Var, numj

In [9]:
def CalProx(x, xi_mean, yi_Var):
    eps = 0.001 
    log = 0
    if yi_Var == 0: 
        numerator = 1
        denominator = 1
    else:
        numerator = np.exp(-(x - xi_mean + eps) ** 2 / (2 * yi_Var))
        denominator = np.sqrt(2 * np.pi * yi_Var )
    m = numerator/denominator
    if m != 0 :
        log = np.log(numerator/denominator)
    return log

In [40]:
def predict(test_data, test_label, mean, var, pyj):
    num,row,col = test_data.shape
    acc = 0
    for i in range(num):
        p = np.log(pyj)
        testdata = test_data[i]
        for j in range(row): 
            for k in range(col):
                xk = testdata[j][k] 
                p[0] = p[0] + CalProx(xk,mean.T[0][j][k],var.T[0][j][k])
                p[1] = p[1] + CalProx(xk,mean.T[1][j][k],var.T[1][j][k])
                p[2] = p[2] + CalProx(xk,mean.T[2][j][k],var.T[2][j][k])
                p[3] = p[3] + CalProx(xk,mean.T[3][j][k],var.T[3][j][k])
                p[4] = p[4] + CalProx(xk,mean.T[4][j][k],var.T[4][j][k])
                p[5] = p[5] + CalProx(xk,mean.T[5][j][k],var.T[5][j][k])
                p[6] = p[6] + CalProx(xk,mean.T[6][j][k],var.T[6][j][k])
                p[7] = p[7] + CalProx(xk,mean.T[7][j][k],var.T[7][j][k])
                p[8] = p[8] + CalProx(xk,mean.T[8][j][k],var.T[8][j][k])
                p[9] = p[9] + CalProx(xk,mean.T[9][j][k],var.T[9][j][k])
        p = np.argmax(p)
        acc = acc + (p == test_label[i])
        #print('real is: ',test_label[i],'  predict is: ',p)
    print('Test accuracy is: ', acc/num)

In [41]:
def Guass_beforeNorm(): # Accuracy without normalization
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    imagefile3 = 't10k-images-idx3-ubyte.gz'
    labelfile3 = 't10k-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_data, test_label = get_labeled_data(imagefile3, labelfile3)
    mean, pyj, pyjk = getMean(train_data,train_label)
    Var, numj = getVar(train_data, train_label, mean)
    predict(test_data, test_label, mean, Var, pyj)

In [29]:
Guass_beforeNorm()

Test accuracy is:  [0.6421]


In [34]:
def Guass_afterNorm():
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    imagefile3 = 't10k-images-idx3-ubyte.gz'
    labelfile3 = 't10k-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_data, test_label = get_labeled_data(imagefile3, labelfile3)
    train1 = normalize(train_data)
    test1 = normalize(test_data)
    Mean, Pyj, pyjk = getMean(train1,train_label)
    var, numj = getVar(train1, train_label, Mean)
    predict(test1, test_label, Mean, var, Pyj)

In [35]:
Guass_afterNorm()

Test accuracy is:  [0.778]


In [36]:
def Guass_scratchwithoutNorm():
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    imagefile3 = 't10k-images-idx3-ubyte.gz'
    labelfile3 = 't10k-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_data, test_label = get_labeled_data(imagefile3, labelfile3)
    train = resize(train_data)
    test = resize(test_data)
    mean, pyj, pyjk = getMean(train,train_label)
    Var, numj = getVar(train, train_label, mean)
    predict(test, test_label, mean, Var, pyj)

In [37]:
Guass_scratchwithoutNorm()

Test accuracy is:  [0.8334]


In [38]:
def Guass_scratchwithNorm():
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    imagefile3 = 't10k-images-idx3-ubyte.gz'
    labelfile3 = 't10k-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_data, test_label = get_labeled_data(imagefile3, labelfile3)
    train = resize(train_data)
    test = resize(test_data)
    train1 = normalize(train)
    test1 = normalize(test)
    mean, pyj, pyjk = getMean(train1,train_label)
    Var, numj = getVar(train1, train_label, mean)
    predict(test1, test_label, mean, Var, pyj)

In [39]:
Guass_scratchwithNorm()

Test accuracy is:  [0.821]


In [32]:
def Guass_afterNorm_traintest():
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_label = train_label
    train1 = normalize(train_data)
    test1 = train1
    Mean, Pyj, pyjk = getMean(train1,train_label)
    var, numj = getVar(train1, train_label, Mean)
    predict(test1, test_label, Mean, var, Pyj)

In [22]:
Guass_afterNorm_traintest()

Test accuracy is:  [0.77153333]


In [33]:
def Guass_scratchwithNorm_traintest():
    imagefile2 = 'train-images-idx3-ubyte.gz'
    labelfile2 = 'train-labels-idx1-ubyte.gz'
    train_data, train_label = get_labeled_data(imagefile2, labelfile2)
    test_label = train_label
    train = resize(train_data)
    train1 = normalize(train)
    test1 = train1
    mean, pyj, pyjk = getMean(train1,train_label)
    Var, numj = getVar(train1, train_label, mean)
    predict(test1, test_label, mean, Var, pyj)

In [26]:
Guass_scratchwithNorm_traintest()

Test accuracy is:  [0.80648333]
